# 📈 Threshold 변화와 모델 성능 평가

---


#### 🧪 암환자 분류 모델에서의 Threshold

이전 예제에서 우리는 **암인지 아닌지를 분류하는 이진 분류 모델**을 학습했습니다.  
이 모델은 각 환자에 대해 **"양성일 확률"**을 예측합니다.

예를 들어, 어떤 환자에 대해 모델이  
> "이 사람은 0.7 확률로 암일 것 같아요."  
라고 예측했다면, 우리는 이 값을 기준으로  
**양성(암)인지 음성(정상)인지** 를 판단해야 합니다.

이때 기준선이 되는 수치를 **Threshold (임계값)** 이라고 부릅니다.

<hr style="opacity:0.2;">

#### 🎯 Threshold를 바꾸면 무슨 일이 생기나요?

보통은 **0.5 이상이면 양성, 아니면 음성** 으로 분류하지만,  
만약 **0.3만 넘으면 양성으로 판단한다면?**

- 더 많은 환자를 **양성(암)** 이라고 예측하게 됩니다.
- 암 환자를 **놓칠 확률(FN)** 이 줄어들고, **Recall** 이 올라갑니다.
- 대신, 정상인도 암으로 잘못 예측할 확률(FP)이 늘어나서 **Precision** 은 낮아질 수 있습니다.

> 💡 즉, **Threshold를 어떻게 설정하느냐에 따라 모델의 성능 지표가 달라집니다.**  
> 모델의 구조나 파라미터는 그대로인데, **출력 해석 방식만 바꾼 것** 인데도 성능이 바뀌는 거죠.

<hr style="opacity:0.2;">

#### 📌 이 말은 곧…

모델의 성능이 **F1 Score 하나로 완전히 표현되지 않을 수 있다는 의미** 입니다.  
Threshold가 고정돼 있다면 F1 Score는 유효하지만,  
**Threshold를 변화시킬 수 있는 상황에서는 더 넓은 관점의 평가 지표** 가 필요합니다.

<hr style="opacity:0.2;">

#### 📊 그래서 사용하는 두 가지 방법

1. **PR Curve (Precision-Recall Curve)**  
   - 다양한 Threshold에서의 **Precision** 과 **Recall** 값을 그래프로 표현  
   - 특히 **양성 클래스가 중요한 경우** 유용함

2. **ROC Curve (Receiver Operating Characteristic Curve)**  
   - 다양한 Threshold에서의 **TPR(Recall)** 과 **FPR(False Positive Rate)** 의 관계를 표현  
   - 일반적으로 전체적인 분류 성능을 평가할 때 사용

<hr style="opacity:0.2;">

#### 🔍 정리

- Threshold가 달라지면 모델의 출력 해석이 달라지고,  
  성능 지표(Accuracy, Precision, Recall, F1 등)도 달라집니다.
- 모델의 전체적인 성능을 종합적으로 평가하고 싶다면  
  **PR 커브** 와 **ROC 커브** 를 함께 살펴봐야 합니다.

<hr style="opacity:0.2;">

# 코드를 통한 실습

In [16]:
# 사이킷런에서 제공하는 데이터셋을 사용하는 방법을 알아봅니다.
# 사이킷런에서 제공하는 데이터셋은 딕셔너리 형태로 되어 있습니다.
# 데이터셋을 불러오고, 데이터셋의 형태를 확인해봅니다.
# 데이터셋을 불러오는 방법은 datasets 모듈을 불러오고, load_데이터셋명() 함수를 사용합니다.
from sklearn import datasets

from sklearn.model_selection import train_test_split
import numpy as np

iris = datasets.load_iris()
X = iris.data
y = iris.target

print(X.shape)  # 4개의 feature를 가진 150개의 데이터입니다.

(150, 4)


In [17]:
random_state = np.random.RandomState(0) # seed를 고정합니다.
n_samples, n_features = X.shape # 150개의 데이터와 4개의 feature가 있습니다.
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)] # np.c_[]를 사용하여 feature를 추가합니다.
# X는 random_state.randn(n_samples, 200 * n_features)를 추가하게 되면 804개의 feature를 가진 150개의 데이터가 됩니다.

print(X.shape)  # 804개의 feature를 가진 150개의 데이터가 되었습니다.

(150, 804)


In [18]:
#- 0, 1 라벨에 속하는 붓꽃 샘플만 사용하도록 제한합니다.
X_train, X_test, y_train, y_test = train_test_split(X[y < 2], y[y < 2],
                                                    test_size=.5,
                                                    random_state=random_state)

print("훈련, 테스트 셋에 사용된 라벨의 종류: {} ".format( set(y_test)))
print("훈련 데이터 shape   :", X_train.shape)
print("테스트 데이터 shape :", X_test.shape)

훈련, 테스트 셋에 사용된 라벨의 종류: {np.int64(0), np.int64(1)} 
훈련 데이터 shape   : (50, 804)
테스트 데이터 shape : (50, 804)


In [19]:
# 두개의 라벨로 나뉜 데이터를 가지고, SVM(Support Vector Machine)을 사용하여 분류를 해봅니다.
# SVM은 데이터를 분류하는 경계선을 찾아주는 알고리즘입니다.
# SVM은 선형 분류와 비선형 분류를 모두 지원합니다.
# 여기서는 비선형 분류를 위해 kernel='poly'를 사용합니다.
# test 데이터에 대한 정확도를 출력합니다.
from sklearn import svm

classifier = svm.SVC(kernel='poly', random_state=random_state)
classifier.fit(X_train, y_train)

classifier.score(X_test,y_test)

0.48

In [20]:
# kernel='poly' 대신 kernel='linear'를 사용하면 선형 분류를 수행합니다.
# test 데이터에 대한 정확도를 출력합니다.
classifier = svm.SVC(kernel='linear', random_state=random_state)
classifier.fit(X_train, y_train)

classifier.score(X_test,y_test)

0.8

In [21]:
# classifer.predict() 함수를 사용하여 테스트 데이터에 대한 예측을 수행합니다.
classifier.predict(X_test)

array([0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1])

In [22]:
y_test

array([1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0])

In [23]:
y_score = classifier.decision_function(X_test)
print(y_score)

[-0.29512751  0.28798352  0.17635465  0.19056886  0.38391605 -0.30841065
 -0.10084254 -0.23481309  0.18576987 -0.36011033 -0.15726747 -0.25714889
 -0.14979669  0.02063898  0.04509171 -0.17239443  0.07287957 -0.0689103
 -0.13452462 -0.30697712  0.25404241 -0.28916471 -0.52061453  0.25252233
  0.02177777 -0.10980907  0.37468422  0.35303004 -0.6211302  -0.42920064
 -0.14770647  0.00593404 -0.34735296  0.32245409 -0.19439024  0.1288847
 -0.0320947  -0.23008604 -0.10135548 -0.46962186  0.05184235  0.0609688
  0.05632596  0.44769206 -0.38804349  0.24704844  0.16063684  0.0144203
 -0.03136574  0.11179177]


In [24]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = classifier.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
rpt_result = classification_report(y_test, y_pred)
print(rpt_result)

[[20  4]
 [ 6 20]]
              precision    recall  f1-score   support

           0       0.77      0.83      0.80        24
           1       0.83      0.77      0.80        26

    accuracy                           0.80        50
   macro avg       0.80      0.80      0.80        50
weighted avg       0.80      0.80      0.80        50



In [25]:
y_pred_new_threshold = classifier.decision_function(X_test) > -0.1
conf_mat = confusion_matrix(y_test, y_pred_new_threshold)
print(conf_mat)
rpt_result = classification_report(y_test, y_pred_new_threshold)
print(rpt_result)

[[18  6]
 [ 5 21]]
              precision    recall  f1-score   support

           0       0.78      0.75      0.77        24
           1       0.78      0.81      0.79        26

    accuracy                           0.78        50
   macro avg       0.78      0.78      0.78        50
weighted avg       0.78      0.78      0.78        50



In [26]:
y_pred_new_threshold = classifier.decision_function(X_test) > -0.2
conf_mat = confusion_matrix(y_test, y_pred_new_threshold)
print(conf_mat)
rpt_result = classification_report(y_test, y_pred_new_threshold)
print(rpt_result)

[[12 12]
 [ 2 24]]
              precision    recall  f1-score   support

           0       0.86      0.50      0.63        24
           1       0.67      0.92      0.77        26

    accuracy                           0.72        50
   macro avg       0.76      0.71      0.70        50
weighted avg       0.76      0.72      0.71        50



In [27]:
y_pred_new_threshold = classifier.decision_function(X_test) > 0.1
conf_mat = confusion_matrix(y_test, y_pred_new_threshold)
print(conf_mat)
rpt_result = classification_report(y_test, y_pred_new_threshold)
print(rpt_result)

[[23  1]
 [12 14]]
              precision    recall  f1-score   support

           0       0.66      0.96      0.78        24
           1       0.93      0.54      0.68        26

    accuracy                           0.74        50
   macro avg       0.80      0.75      0.73        50
weighted avg       0.80      0.74      0.73        50



In [28]:
y_pred_new_threshold = classifier.decision_function(X_test) > 0.2
conf_mat = confusion_matrix(y_test, y_pred_new_threshold)
print(conf_mat)
rpt_result = classification_report(y_test, y_pred_new_threshold)
print(rpt_result)

[[24  0]
 [17  9]]
              precision    recall  f1-score   support

           0       0.59      1.00      0.74        24
           1       1.00      0.35      0.51        26

    accuracy                           0.66        50
   macro avg       0.79      0.67      0.63        50
weighted avg       0.80      0.66      0.62        50

